# Online Retail Transactions - Cleaning Step

## What this section does

Takes the raw Online Retail invoice lines from `ltv_df.xlsx` and turns them into a clean transaction table that is safe to use for customer value and churn analysis.

**Input**

Each row is an invoice line with:

- `Invoice` - invoice number (cancellations start with `"C"`)
- `StockCode` - product code
- `Description` - product description
- `Quantity` - units sold
- `InvoiceDate` - when the invoice was issued
- `Price` - unit price
- `Customer ID` - customer identifier
- `Country` - customer country

The raw data contains missing customer IDs, cancellations, non-positive quantities/prices, and duplicates.

## What this cleaning step enforces

- **Keeps only valid transactions**
  - Drops rows with missing `Customer ID`
  - Removes cancellations (`Invoice` starting with `"C"`)
  - Filters out rows where `Quantity <= 0` or `Price <= 0`
  - Drops exact duplicate rows

- **Standardizes core fields**
  - Converts `Customer ID` to integer `customer_id`
  - Parses `InvoiceDate` to `invoice_date` (datetime)
  - Renames columns to consistent snake_case (`invoice_no`, `stock_code`, `description`, `quantity`, `unit_price`, `country`)

After this step, `tx_df` is a clean, one-row-per-transaction table ready to roll up to customer level in the next chapter.

In [ ]:
import pandas as pd

#load the raw excel file
ltv_df = pd.read_excel("ltv_df.xlsx", engine="openpyxl")

#structure
print("Shape:", ltv_df.shape)
print("\nDtypes:\n", ltv_df.dtypes)

#nulls per column
print("\nNulls per column:\n", ltv_df.isnull().sum())

#distinct customers and missing customer IDs
if "Customer ID" in ltv_df.columns:
    print("\nDistinct customers:", ltv_df["Customer ID"].nunique())
    print("Rows with missing Customer ID:", ltv_df["Customer ID"].isna().sum())
else:
    print("\nColumn 'Customer ID' not found in columns:", ltv_df.columns.tolist())

#numeric checks for Quantity and Price
num_cols = [c for c in ["Quantity", "Price"] if c in ltv_df.columns]

if num_cols:
    print("\nNumeric summary (Quantity / Price):")
    print(ltv_df[num_cols].describe())

    if "Price" in ltv_df.columns:
        print("\nAny non positive prices:", (ltv_df["Price"] <= 0).any())
    if "Quantity" in ltv_df.columns:
        print("Any zero or negative quantities:", (ltv_df["Quantity"] <= 0).any())
else:
    print("\nNo Quantity / Price columns found. Columns are:", ltv_df.columns.tolist())

#peek
ltv_df.head()

Shape: (525461, 8)

Dtypes:
 Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

Nulls per column:
 Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

Distinct customers: 4383
Rows with missing Customer ID: 107927

Numeric summary (Quantity / Price):
            Quantity          Price
count  525461.000000  525461.000000
mean       10.337667       4.688834
std       107.424110     146.126914
min     -9600.000000  -53594.360000
25%         1.000000       1.250000
50%         3.000000       2.100000
75%        10.000000       4.210000
max     19152.000000   25111.090000

Any non positive prices: True
Any zero or negative quantities: True


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
#create a working copy from the raw data
tx_df = ltv_df.copy()

print("Starting rows:", len(tx_df))

#drop rows with missing Customer ID
before = len(tx_df)
tx_df = tx_df.dropna(subset=["Customer ID"])
print("Dropped with missing Customer ID:", before - len(tx_df))

#remove cancellation rows (Invoices starting with 'C')
before = len(tx_df)
tx_df = tx_df[~tx_df["Invoice"].astype(str).str.startswith("C")]
print("Dropped cancellation rows:", before - len(tx_df))

#remove rows with non-positive Quantity or Price
before = len(tx_df)
tx_df = tx_df[(tx_df["Quantity"] > 0) & (tx_df["Price"] > 0)]
print("Dropped non-positive Quantity/Price rows:", before - len(tx_df))

#drop exact duplicate rows
before = len(tx_df)
tx_df = tx_df.drop_duplicates()
print("Dropped duplicate rows:", before - len(tx_df))

#type fixes and renaming
tx_df["customer_id"] = tx_df["Customer ID"].astype("int64")
tx_df["invoice_date"] = pd.to_datetime(tx_df["InvoiceDate"])

tx_df = tx_df.rename(
    columns={
        "Invoice": "invoice_no",
        "StockCode": "stock_code",
        "Description": "description",
        "Quantity": "quantity",
        "Price": "unit_price",
        "Country": "country",
    }
)

#drop old columns with wierd names
tx_df = tx_df.drop(columns=["Customer ID", "InvoiceDate"])

#final check
print("\nCleaned shape:", tx_df.shape)
print("\nNulls per column after cleaning:\n", tx_df.isnull().sum())

tx_df.head()

Starting rows: 525461
Dropped with missing Customer ID: 107927
Dropped cancellation rows: 9839
Dropped non-positive Quantity/Price rows: 31
Dropped duplicate rows: 6748

Cleaned shape: (400916, 8)

Nulls per column after cleaning:
 invoice_no      0
stock_code      0
description     0
quantity        0
unit_price      0
country         0
customer_id     0
invoice_date    0
dtype: int64


,invoice_no,stock_code,description,quantity,unit_price,country,customer_id,invoice_date
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,United Kingdom,13085,2009-12-01 07:45:00
1,489434,79323P,PINK CHERRY LIGHTS,12,6.75,United Kingdom,13085,2009-12-01 07:45:00
2,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,United Kingdom,13085,2009-12-01 07:45:00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,United Kingdom,13085,2009-12-01 07:45:00
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,United Kingdom,13085,2009-12-01 07:45:00


In [ ]:
#save the result
tx_df.to_csv("ltv_clean.csv", index=False)

## Conclusion

The transaction data is now fully usable for customer value and churn work.  
All rows have valid customer IDs, positive quantities and prices, no cancellations, and no duplicates.  
From here we can safely aggregate to customer level and build the CLTV and churn metrics.